In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pickle
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts

# Sklearn
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

# Set random seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

✅ Setup complete


In [ ]:
class TrajectoryTransformer(nn.Module):
    """
    Advanced Transformer model for multi-agent trajectory prediction
    with attention mechanisms and uncertainty estimation
    """
    def __init__(self, 
                 input_dim=64,           # Feature dimension per player
                 hidden_dim=256,         # Hidden layer size
                 num_heads=8,            # Attention heads
                 num_layers=6,           # Transformer layers
                 max_players=22,         # Max players on field
                 max_seq_len=50,         # Max input frames
                 max_output_frames=35,   # Max frames to predict
                 dropout=0.1):
        super().__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.max_output_frames = max_output_frames
        
        # Input projection
        self.input_projection = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout)
        )
        
        # Positional encoding for temporal information
        self.temporal_encoding = nn.Parameter(
            self._generate_positional_encoding(max_seq_len, hidden_dim)
        )
        
        # Player role embedding (Passer, Receiver, Route Runner, Coverage, Other)
        self.role_embedding = nn.Embedding(5, hidden_dim)
        
        # Side embedding (Offense/Defense)
        self.side_embedding = nn.Embedding(2, hidden_dim)
        
        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim,
            nhead=num_heads,
            dim_feedforward=hidden_dim * 4,
            dropout=dropout,
            activation='gelu',
            batch_first=True,
            norm_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer, 
            num_layers=num_layers
        )
        
        # Cross-attention for player interactions
        self.cross_attention = nn.MultiheadAttention(
            embed_dim=hidden_dim,
            num_heads=num_heads,
            dropout=dropout,
            batch_first=True
        )
        
        # Trajectory decoder with residual connections
        self.trajectory_decoder = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.LayerNorm(hidden_dim),
                nn.GELU(),
                nn.Dropout(dropout)
            ) for _ in range(3)
        ])
        
        # Output head - predicts variable length trajectories
        self.output_head = nn.Linear(hidden_dim, max_output_frames * 2)
        
        # Uncertainty estimation head
        self.uncertainty_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.GELU(),
            nn.Linear(hidden_dim // 2, max_output_frames * 2)
        )
        
        # Frame-specific output projection
        self.frame_predictor = nn.Linear(hidden_dim, 2)
        
    def _generate_positional_encoding(self, max_len, d_model):
        """Generate sinusoidal positional encoding"""
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-np.log(10000.0) / d_model))
        pe = torch.zeros(1, max_len, d_model)
        pe[0, :, 0::2] = torch.sin(position * div_term)
        pe[0, :, 1::2] = torch.cos(position * div_term)
        return pe
        
    def forward(self, x, role_ids, side_ids, num_output_frames, mask=None):
        """
        Args:
            x: [batch, seq_len, num_players, input_dim]
            role_ids: [batch, num_players] - player role IDs (0-4)
            side_ids: [batch, num_players] - player side IDs (0=offense, 1=defense)
            num_output_frames: [batch] - number of frames to predict per sample
            mask: [batch, seq_len, num_players] - attention mask
        Returns:
            trajectories: [batch, num_players, max_output_frames, 2]
            uncertainties: [batch, num_players, max_output_frames, 2]
        """
        batch_size, seq_len, num_players, _ = x.shape
        
        # Reshape for processing: [B*P, T, D]
        x = x.reshape(batch_size * num_players, seq_len, self.input_dim)
        
        # Project input
        x = self.input_projection(x)
        
        # Add temporal encoding
        x = x + self.temporal_encoding[:, :seq_len, :]
        
        # Add role embedding
        role_emb = self.role_embedding(role_ids)  # [B, P, H]
        role_emb = role_emb.reshape(batch_size * num_players, 1, self.hidden_dim)
        x = x + role_emb
        
        # Add side embedding
        side_emb = self.side_embedding(side_ids)  # [B, P, H]
        side_emb = side_emb.reshape(batch_size * num_players, 1, self.hidden_dim)
        x = x + side_emb
        
        # Apply transformer encoder
        if mask is not None:
            mask = mask.reshape(batch_size * num_players, seq_len)
        encoded = self.transformer_encoder(x, src_key_padding_mask=mask)
        
        # Reshape back for cross-attention: [B, P, T, H]
        encoded = encoded.reshape(batch_size, num_players, seq_len, self.hidden_dim)
        
        # Apply cross-attention across players (at last timestep)
        last_encoded = encoded[:, :, -1, :]  # [B, P, H]
        attended, _ = self.cross_attention(last_encoded, last_encoded, last_encoded)
        
        # Residual connection
        last_encoded = last_encoded + attended
        
        # Apply decoder layers with residual connections
        hidden = last_encoded
        for decoder_layer in self.trajectory_decoder:
            hidden = hidden + decoder_layer(hidden)
        
        # Predict trajectories
        traj_flat = self.output_head(hidden)  # [B, P, max_T*2]
        trajectories = traj_flat.reshape(
            batch_size, num_players, self.max_output_frames, 2
        )
        
        # Predict uncertainties (log variance)
        uncert_flat = self.uncertainty_head(hidden)
        uncertainties = F.softplus(uncert_flat).reshape(
            batch_size, num_players, self.max_output_frames, 2
        )
        
        return trajectories, uncertainties

print("✅ Transformer model defined")

In [ ]:
class NFLTrajectoryDataset(Dataset):
    """
    Dataset for loading NFL tracking data for trajectory prediction
    Handles variable sequence lengths and player counts
    """
    def __init__(self, 
                 weeks,  # List of week numbers to include
                 data_dir='.',
                 supplementary_file='supplementary_data.csv',
                 max_input_frames=40,
                 max_players=22,
                 normalize=True,
                 augment=False):
        
        self.max_input_frames = max_input_frames
        self.max_players = max_players
        self.normalize = normalize
        self.augment = augment
        
        print(f"Loading data for weeks: {weeks}")
        
        # Load input data
        input_files = [f'input_2023_w{w:02d}.csv' for w in weeks]
        self.input_data = pd.concat([
            pd.read_csv(f) for f in input_files if Path(f).exists()
        ], ignore_index=True)
        
        # Load output data
        output_files = [f'output_2023_w{w:02d}.csv' for w in weeks]
        self.output_data = pd.concat([
            pd.read_csv(f) for f in output_files if Path(f).exists()
        ], ignore_index=True)
        
        # Load supplementary data
        self.supplementary = pd.read_csv(supplementary_file)
        
        print(f"  Input data: {len(self.input_data):,} rows")
        print(f"  Output data: {len(self.output_data):,} rows")
        
        # Get unique plays
        self.plays = self.input_data.groupby(['game_id', 'play_id']).size().reset_index()[['game_id', 'play_id']]
        print(f"  Total plays: {len(self.plays):,}")
        
        # Define feature columns
        self.feature_cols = [
            'x', 'y', 's', 'a', 'dir', 'o',
            'absolute_yardline_number'
        ]
        
        # Add engineered features if they exist
        engineered_features = [
            'distance_to_ball_land', 'angle_to_ball_land',
            'velocity_x', 'velocity_y',
            'distance_to_qb', 'distance_to_los'
        ]
        for feat in engineered_features:
            if feat in self.input_data.columns:
                self.feature_cols.append(feat)
        
        print(f"  Using {len(self.feature_cols)} features: {self.feature_cols}")
        
        # Compute normalization stats
        if self.normalize:
            self.compute_normalization_stats()
        
        # Role mapping
        self.role_map = {
            'Passer': 0,
            'Targeted Receiver': 1,
            'Other Route Runner': 2,
            'Defensive Coverage': 3
        }
        
        # Side mapping
        self.side_map = {
            'Offense': 0,
            'Defense': 1
        }
    
    def compute_normalization_stats(self):
        """Compute mean and std for normalization"""
        print("  Computing normalization statistics...")
        self.feature_means = {}
        self.feature_stds = {}
        
        for col in self.feature_cols:
            if col in self.input_data.columns:
                self.feature_means[col] = self.input_data[col].mean()
                self.feature_stds[col] = self.input_data[col].std()
            else:
                self.feature_means[col] = 0.0
                self.feature_stds[col] = 1.0
        
        print("  ✅ Normalization stats computed")
    
    def __len__(self):
        return len(self.plays)
    
    def __getitem__(self, idx):
        """
        Returns a single play with input and output sequences
        """
        game_id = self.plays.iloc[idx]['game_id']
        play_id = self.plays.iloc[idx]['play_id']
        
        # Get input data for this play
        input_play = self.input_data[
            (self.input_data['game_id'] == game_id) & 
            (self.input_data['play_id'] == play_id)
        ].copy()
        
        # Get output data for this play
        output_play = self.output_data[
            (self.output_data['game_id'] == game_id) & 
            (self.output_data['play_id'] == play_id)
        ].copy()
        
        # Sort by frame and player
        input_play = input_play.sort_values(['frame_id', 'nfl_id'])
        output_play = output_play.sort_values(['frame_id', 'nfl_id'])
        
        # Get unique frames and players
        frames = sorted(input_play['frame_id'].unique())
        players = sorted(input_play['nfl_id'].unique())
        
        num_frames = min(len(frames), self.max_input_frames)
        num_players = min(len(players), self.max_players)
        
        # Initialize tensors
        input_seq = np.zeros((num_frames, num_players, len(self.feature_cols)))
        mask = np.ones((num_frames, num_players), dtype=bool)
        
        # Role and side IDs
        role_ids = np.zeros(num_players, dtype=np.int64)
        side_ids = np.zeros(num_players, dtype=np.int64)
        players_to_predict = np.zeros(num_players, dtype=bool)
        
        # Fill input sequence
        for f_idx, frame in enumerate(frames[:num_frames]):
            frame_data = input_play[input_play['frame_id'] == frame]
            
            for p_idx, player in enumerate(players[:num_players]):
                player_data = frame_data[frame_data['nfl_id'] == player]
                
                if len(player_data) > 0:
                    row = player_data.iloc[0]
                    
                    # Extract features
                    for feat_idx, col in enumerate(self.feature_cols):
                        if col in player_data.columns:
                            input_seq[f_idx, p_idx, feat_idx] = row[col]
                    
                    mask[f_idx, p_idx] = False
                    
                    # Get role and side (only need to do once)
                    if f_idx == 0:
                        role = row.get('player_role', 'Other')
                        role_ids[p_idx] = self.role_map.get(role, 3)
                        
                        side = row.get('player_side', 'Defense')
                        side_ids[p_idx] = self.side_map.get(side, 1)
                        
                        players_to_predict[p_idx] = row.get('player_to_predict', False)
        
        # Normalize features
        if self.normalize:
            for feat_idx, col in enumerate(self.feature_cols):
                mean = self.feature_means.get(col, 0.0)
                std = self.feature_stds.get(col, 1.0)
                input_seq[:, :, feat_idx] = (input_seq[:, :, feat_idx] - mean) / (std + 1e-8)
        
        # Data augmentation (flip field)
        if self.augment and np.random.rand() > 0.5:
            # Flip y-coordinates
            y_idx = self.feature_cols.index('y')
            input_seq[:, :, y_idx] = 53.3 - input_seq[:, :, y_idx]
            
            # Flip angles
            if 'dir' in self.feature_cols:
                dir_idx = self.feature_cols.index('dir')
                input_seq[:, :, dir_idx] = (360 - input_seq[:, :, dir_idx]) % 360
            if 'o' in self.feature_cols:
                o_idx = self.feature_cols.index('o')
                input_seq[:, :, o_idx] = (360 - input_seq[:, :, o_idx]) % 360
        
        # Extract output trajectories
        output_frames = sorted(output_play['frame_id'].unique())
        num_output_frames = len(output_frames)
        
        # Initialize output with zeros (will be masked during loss computation)
        output_seq = np.zeros((num_players, 35, 2))  # Max 35 frames
        
        for f_idx, frame in enumerate(output_frames):
            frame_data = output_play[output_play['frame_id'] == frame]
            
            for p_idx, player in enumerate(players[:num_players]):
                player_data = frame_data[frame_data['nfl_id'] == player]
                
                if len(player_data) > 0:
                    output_seq[p_idx, f_idx, 0] = player_data.iloc[0]['x']
                    output_seq[p_idx, f_idx, 1] = player_data.iloc[0]['y']
        
        # Get supplementary info
        supp = self.supplementary[
            (self.supplementary['game_id'] == game_id) & 
            (self.supplementary['play_id'] == play_id)
        ]
        
        metadata = {
            'game_id': game_id,
            'play_id': play_id,
            'num_output_frames': num_output_frames,
            'num_players': num_players,
            'num_frames': num_frames
        }
        
        if len(supp) > 0:
            supp_row = supp.iloc[0]
            metadata['pass_result'] = supp_row.get('pass_result', 'Unknown')
            metadata['coverage_type'] = supp_row.get('team_coverage_type', 'Unknown')
        
        return {
            'input_seq': torch.FloatTensor(input_seq),
            'output_seq': torch.FloatTensor(output_seq),
            'mask': torch.BoolTensor(mask),
            'role_ids': torch.LongTensor(role_ids),
            'side_ids': torch.LongTensor(side_ids),
            'players_to_predict': torch.BoolTensor(players_to_predict),
            'num_output_frames': num_output_frames,
            'metadata': metadata
        }

print("✅ Dataset class defined")

In [ ]:
# Define week splits for cross-validation
# Strategy: Use weeks as folds to maintain temporal structure

# Option 1: Simple train/val split
train_weeks = list(range(1, 15))  # Weeks 1-14 for training
val_weeks = [15, 16, 17, 18]      # Weeks 15-18 for validation

print("Creating datasets...")
train_dataset = NFLTrajectoryDataset(
    weeks=train_weeks,
    normalize=True,
    augment=True  # Enable augmentation for training
)

val_dataset = NFLTrajectoryDataset(
    weeks=val_weeks,
    normalize=True,
    augment=False  # No augmentation for validation
)

# Copy normalization stats from train to val
val_dataset.feature_means = train_dataset.feature_means
val_dataset.feature_stds = train_dataset.feature_stds

print(f"\n📊 Dataset Summary:")
print(f"  Training plays: {len(train_dataset):,}")
print(f"  Validation plays: {len(val_dataset):,}")
print(f"  Total plays: {len(train_dataset) + len(val_dataset):,}")

# Create data loaders
batch_size = 16  # Adjust based on GPU memory
num_workers = 4  # Adjust based on CPU cores

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True if torch.cuda.is_available() else False,
    drop_last=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True if torch.cuda.is_available() else False
)

print(f"\n📦 DataLoader Summary:")
print(f"  Training batches: {len(train_loader):,}")
print(f"  Validation batches: {len(val_loader):,}")
print(f"  Batch size: {batch_size}")

# Test loading a batch
print("\n🧪 Testing data loading...")
test_batch = next(iter(train_loader))
print(f"  Input shape: {test_batch['input_seq'].shape}")
print(f"  Output shape: {test_batch['output_seq'].shape}")
print(f"  Role IDs shape: {test_batch['role_ids'].shape}")
print(f"  ✅ Data loading successful!")

In [ ]:
class TrajectoryLoss(nn.Module):
    """
    Custom loss function for trajectory prediction
    Combines MSE with uncertainty weighting
    """
    def __init__(self, uncertainty_weight=0.1):
        super().__init__()
        self.uncertainty_weight = uncertainty_weight
    
    def forward(self, pred_traj, pred_uncert, target_traj, players_to_predict, num_output_frames):
        """
        Args:
            pred_traj: [B, P, T, 2] - predicted trajectories
            pred_uncert: [B, P, T, 2] - predicted uncertainties
            target_traj: [B, P, T, 2] - ground truth trajectories
            players_to_predict: [B, P] - mask for which players to evaluate
            num_output_frames: [B] - actual number of frames per sample
        """
        batch_size = pred_traj.shape[0]
        total_loss = 0.0
        
        for b in range(batch_size):
            # Get actual frames for this sample
            T = num_output_frames[b]
            
            # Get players to predict
            player_mask = players_to_predict[b]
            
            if player_mask.sum() == 0:
                continue
            
            # Extract relevant predictions and targets
            pred = pred_traj[b, player_mask, :T, :]
            uncert = pred_uncert[b, player_mask, :T, :]
            target = target_traj[b, player_mask, :T, :]
            
            # Compute MSE loss
            mse_loss = F.mse_loss(pred, target)
            
            # Uncertainty-weighted loss (negative log likelihood)
            # loss = 0.5 * (error^2 / variance + log(variance))
            error_sq = (pred - target) ** 2
            nll_loss = 0.5 * (error_sq / (uncert + 1e-6) + torch.log(uncert + 1e-6))
            nll_loss = nll_loss.mean()
            
            # Combine losses
            total_loss += mse_loss + self.uncertainty_weight * nll_loss
        
        return total_loss / batch_size

print("✅ Loss function defined")

In [ ]:
def train_epoch(model, train_loader, optimizer, criterion, device, epoch):
    """Train for one epoch"""
    model.train()
    total_loss = 0.0
    num_batches = 0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch} [Train]")
    
    for batch in pbar:
        # Move data to device
        input_seq = batch['input_seq'].to(device)
        output_seq = batch['output_seq'].to(device)
        mask = batch['mask'].to(device)
        role_ids = batch['role_ids'].to(device)
        side_ids = batch['side_ids'].to(device)
        players_to_predict = batch['players_to_predict'].to(device)
        num_output_frames = batch['num_output_frames']
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        pred_traj, pred_uncert = model(
            input_seq, 
            role_ids, 
            side_ids, 
            num_output_frames,
            mask=mask
        )
        
        # Compute loss
        loss = criterion(
            pred_traj, 
            pred_uncert, 
            output_seq, 
            players_to_predict,
            num_output_frames
        )
        
        # Backward pass
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        # Update weights
        optimizer.step()
        
        # Track loss
        total_loss += loss.item()
        num_batches += 1
        
        # Update progress bar
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    return total_loss / num_batches


def validate_epoch(model, val_loader, criterion, device, epoch):
    """Validate for one epoch"""
    model.eval()
    total_loss = 0.0
    num_batches = 0
    
    all_predictions = []
    all_targets = []
    all_metadata = []
    
    pbar = tqdm(val_loader, desc=f"Epoch {epoch} [Val]")
    
    with torch.no_grad():
        for batch in pbar:
            # Move data to device
            input_seq = batch['input_seq'].to(device)
            output_seq = batch['output_seq'].to(device)
            mask = batch['mask'].to(device)
            role_ids = batch['role_ids'].to(device)
            side_ids = batch['side_ids'].to(device)
            players_to_predict = batch['players_to_predict'].to(device)
            num_output_frames = batch['num_output_frames']
            
            # Forward pass
            pred_traj, pred_uncert = model(
                input_seq, 
                role_ids, 
                side_ids, 
                num_output_frames,
                mask=mask
            )
            
            # Compute loss
            loss = criterion(
                pred_traj, 
                pred_uncert, 
                output_seq, 
                players_to_predict,
                num_output_frames
            )
            
            # Track loss
            total_loss += loss.item()
            num_batches += 1
            
            # Store predictions for analysis
            all_predictions.append(pred_traj.cpu())
            all_targets.append(output_seq.cpu())
            all_metadata.append(batch['metadata'])
            
            # Update progress bar
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    return total_loss / num_batches, all_predictions, all_targets, all_metadata


def train_model(model, train_loader, val_loader, num_epochs=50, lr=1e-4, device='cuda'):
    """
    Complete training loop with validation
    """
    print(f"\n🚀 Starting training for {num_epochs} epochs")
    print(f"   Device: {device}")
    print(f"   Learning rate: {lr}")
    
    # Move model to device
    model = model.to(device)
    
    # Optimizer
    optimizer = AdamW(
        model.parameters(), 
        lr=lr, 
        weight_decay=1e-5,
        betas=(0.9, 0.999)
    )
    
    # Learning rate scheduler
    scheduler = ReduceLROnPlateau(
        optimizer, 
        mode='min', 
        factor=0.5, 
        patience=5, 
        verbose=True,
        min_lr=1e-7
    )
    
    # Loss function
    criterion = TrajectoryLoss(uncertainty_weight=0.1)
    
    # Training history
    history = {
        'train_loss': [],
        'val_loss': [],
        'learning_rates': []
    }
    
    best_val_loss = float('inf')
    patience_counter = 0
    max_patience = 10
    
    # Training loop
    for epoch in range(1, num_epochs + 1):
        # Train
        train_loss = train_epoch(model, train_loader, optimizer, criterion, device, epoch)
        
        # Validate
        val_loss, val_preds, val_targets, val_metadata = validate_epoch(
            model, val_loader, criterion, device, epoch
        )
        
        # Update scheduler
        scheduler.step(val_loss)
        
        # Get current learning rate
        current_lr = optimizer.param_groups[0]['lr']
        
        # Save history
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['learning_rates'].append(current_lr)
        
        # Print epoch summary
        print(f"\n{'='*60}")
        print(f"Epoch {epoch}/{num_epochs}")
        print(f"  Train Loss: {train_loss:.6f}")
        print(f"  Val Loss:   {val_loss:.6f}")
        print(f"  LR:         {current_lr:.2e}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
                'history': history
            }, 'best_model.pth')
            
            print(f"  ✅ New best model saved! (Val Loss: {val_loss:.6f})")
        else:
            patience_counter += 1
            print(f"  No improvement ({patience_counter}/{max_patience})")
        
        # Early stopping
        if patience_counter >= max_patience:
            print(f"\n⚠️ Early stopping triggered after {epoch} epochs")
            break
        
        print(f"{'='*60}\n")
    
    print("\n✅ Training complete!")
    print(f"   Best validation loss: {best_val_loss:.6f}")
    
    return history

print("✅ Training functions defined")

In [ ]:
# Model hyperparameters
model_config = {
    'input_dim': len(train_dataset.feature_cols),
    'hidden_dim': 256,
    'num_heads': 8,
    'num_layers': 6,
    'max_players': 22,
    'max_seq_len': 40,
    'max_output_frames': 35,
    'dropout': 0.1
}

print("🏗️ Initializing model...")
print(f"   Config: {model_config}")

model = TrajectoryTransformer(**model_config)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")

# Train the model
history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=50,
    lr=1e-4,
    device=device
)

# Save training history
with open('training_history.pkl', 'wb') as f:
    pickle.dump(history, f)

print("\n✅ Model training complete!")

In [ ]:
def train_ensemble(num_models=5, train_loader=None, val_loader=None, device='cuda'):
    """
    Train multiple models with different initializations for ensemble
    """
    print(f"\n🎯 Training ensemble of {num_models} models")
    
    ensemble_models = []
    ensemble_histories = []
    
    for i in range(num_models):
        print(f"\n{'='*60}")
        print(f"Training Model {i+1}/{num_models}")
        print(f"{'='*60}")
        
        # Set different seed for each model
        set_seed(42 + i)
        
        # Initialize model
        model = TrajectoryTransformer(**model_config)
        
        # Train model
        history = train_model(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            num_epochs=30,  # Fewer epochs per model
            lr=1e-4,
            device=device
        )
        
        # Load best checkpoint
        checkpoint = torch.load('best_model.pth')
        model.load_state_dict(checkpoint['model_state_dict'])
        
        # Save model
        torch.save(model.state_dict(), f'ensemble_model_{i+1}.pth')
        
        ensemble_models.append(model)
        ensemble_histories.append(history)
        
        print(f"✅ Model {i+1} complete!")
    
    return ensemble_models, ensemble_histories


def ensemble_predict(models, input_seq, role_ids, side_ids, num_output_frames, mask=None):
    """
    Make predictions using ensemble of models
    """
    all_predictions = []
    all_uncertainties = []
    
    for model in models:
        model.eval()
        with torch.no_grad():
            pred_traj, pred_uncert = model(
                input_seq, role_ids, side_ids, num_output_frames, mask
            )
            all_predictions.append(pred_traj)
            all_uncertainties.append(pred_uncert)
    
    # Average predictions
    ensemble_pred = torch.stack(all_predictions).mean(dim=0)
    
    # Combine uncertainties (epistemic + aleatoric)
    mean_uncert = torch.stack(all_uncertainties).mean(dim=0)
    pred_variance = torch.stack(all_predictions).var(dim=0)
    ensemble_uncert = mean_uncert + pred_variance
    
    return ensemble_pred, ensemble_uncert


# Train ensemble 
ensemble_models, ensemble_histories = train_ensemble(
    num_models=5,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device
)

print("✅ Ensemble training functions defined")

In [ ]:
class PhysicsConstraints:
    """
    Apply physics-based constraints to trajectory predictions
    """
    def __init__(self, 
                 max_speed=12.0,        # yards/second (elite NFL speed ~23 mph)
                 max_acceleration=8.0,   # yards/second^2
                 dt=0.1):                # time step (10 Hz tracking)
        self.max_speed = max_speed
        self.max_acceleration = max_acceleration
        self.dt = dt
    
    def apply_constraints(self, trajectories, initial_velocity=None):
        """
        Apply physics constraints to predicted trajectories
        
        Args:
            trajectories: [batch, num_players, num_frames, 2] - predicted (x, y)
            initial_velocity: [batch, num_players, 2] - initial velocity (vx, vy)
        
        Returns:
            constrained_trajectories: [batch, num_players, num_frames, 2]
        """
        batch_size, num_players, num_frames, _ = trajectories.shape
        
        constrained = trajectories.clone()
        
        for b in range(batch_size):
            for p in range(num_players):
                # Get trajectory for this player
                traj = constrained[b, p]  # [T, 2]
                
                # Initialize velocity
                if initial_velocity is not None:
                    velocity = initial_velocity[b, p].clone()
                else:
                    velocity = torch.zeros(2)
                
                # Apply constraints frame by frame
                for t in range(1, num_frames):
                    # Compute displacement
                    displacement = traj[t] - traj[t-1]
                    
                    # Compute implied velocity
                    implied_velocity = displacement / self.dt
                    
                    # Compute implied acceleration
                    acceleration = (implied_velocity - velocity) / self.dt
                    
                    # Constrain acceleration
                    accel_magnitude = torch.norm(acceleration)
                    if accel_magnitude > self.max_acceleration:
                        acceleration = acceleration * (self.max_acceleration / accel_magnitude)
                    
                    # Update velocity
                    velocity = velocity + acceleration * self.dt
                    
                    # Constrain speed
                    speed = torch.norm(velocity)
                    if speed > self.max_speed:
                        velocity = velocity * (self.max_speed / speed)
                    
                    # Update position
                    constrained[b, p, t] = traj[t-1] + velocity * self.dt
                    
                    # Field boundaries (0-120 yards x, 0-53.3 yards y)
                    constrained[b, p, t, 0] = torch.clamp(constrained[b, p, t, 0], 0, 120)
                    constrained[b, p, t, 1] = torch.clamp(constrained[b, p, t, 1], 0, 53.3)
        
        return constrained
    
    def smooth_trajectory(self, trajectories, window_size=3):
        """
        Apply moving average smoothing to trajectories
        """
        batch_size, num_players, num_frames, coords = trajectories.shape
        
        smoothed = trajectories.clone()
        
        for b in range(batch_size):
            for p in range(num_players):
                for c in range(coords):
                    # Apply 1D convolution for smoothing
                    signal = trajectories[b, p, :, c]
                    kernel = torch.ones(window_size) / window_size
                    
                    # Pad signal
                    padded = F.pad(signal.unsqueeze(0).unsqueeze(0), 
                                   (window_size//2, window_size//2), 
                                   mode='replicate')
                    
                    # Convolve
                    smoothed_signal = F.conv1d(padded, 
                                               kernel.unsqueeze(0).unsqueeze(0))
                    
                    smoothed[b, p, :, c] = smoothed_signal.squeeze()
        
        return smoothed


# Initialize physics constraints
physics = PhysicsConstraints(
    max_speed=12.0,
    max_acceleration=8.0,
    dt=0.1
)

print("✅ Physics constraints defined")

In [ ]:
class ContextAwarePredictor:
    """
    Wrapper that incorporates play context into predictions
    """
    def __init__(self, model, physics_constraints, supplementary_data):
        self.model = model
        self.physics = physics_constraints
        self.supplementary = supplementary_data
        
        # Coverage type embeddings (learned or rule-based adjustments)
        self.coverage_adjustments = {
            'Cover 0': 1.0,   # Man coverage - tighter coverage
            'Cover 1': 0.95,
            'Cover 2': 0.9,   # Zone coverage - more space
            'Cover 3': 0.85,
            'Cover 4': 0.8,
            'Cover 6': 0.85,
            'Man': 1.0,
            'Zone': 0.85,
            'Prevent': 0.7    # Deep zones - lots of space
        }
    
    def predict_with_context(self, batch, device='cuda'):
        """
        Make context-aware predictions
        """
        # Get model predictions
        input_seq = batch['input_seq'].to(device)
        role_ids = batch['role_ids'].to(device)
        side_ids = batch['side_ids'].to(device)
        num_output_frames = batch['num_output_frames']
        mask = batch['mask'].to(device)
        
        self.model.eval()
        with torch.no_grad():
            pred_traj, pred_uncert = self.model(
                input_seq, role_ids, side_ids, num_output_frames, mask
            )
        
        # Apply physics constraints
        pred_traj = self.physics.apply_constraints(pred_traj)
        pred_traj = self.physics.smooth_trajectory(pred_traj)
        
        # Apply context-based adjustments
        for b in range(len(batch['metadata'])):
            metadata = batch['metadata'][b]
            game_id = metadata['game_id']
            play_id = metadata['play_id']
            
            # Get play context
            play_info = self.supplementary[
                (self.supplementary['game_id'] == game_id) & 
                (self.supplementary['play_id'] == play_id)
            ]
            
            if len(play_info) > 0:
                coverage = play_info.iloc[0].get('team_coverage_type', 'Unknown')
                
                # Adjust predictions based on coverage
                if coverage in self.coverage_adjustments:
                    adjustment = self.coverage_adjustments[coverage]
                    
                    # Scale defensive player movements
                    defense_mask = side_ids[b] == 1  # Defense
                    pred_traj[b, defense_mask] *= adjustment
        
        return pred_traj, pred_uncert
    
    def predict_with_route_awareness(self, batch, route_patterns, device='cuda'):
        """
        Incorporate route pattern knowledge
        """
        # Get base predictions
        pred_traj, pred_uncert = self.predict_with_context(batch, device)
        
        # Adjust receiver trajectories based on route patterns
        for b in range(len(batch['metadata'])):
            metadata = batch['metadata'][b]
            
            # Get targeted receiver
            role_ids = batch['role_ids'][b]
            targeted_receiver_mask = (role_ids == 1)  # Targeted Receiver
            
            if targeted_receiver_mask.any():
                # Apply route-specific adjustments
                # This could be learned or rule-based
                pass
        
        return pred_traj, pred_uncert


# Initialize context-aware predictor
context_predictor = ContextAwarePredictor(
    model=model,
    physics_constraints=physics,
    supplementary_data=train_dataset.supplementary
)

print("✅ Context-aware predictor defined")

In [ ]:
def compute_metrics(predictions, targets, players_to_predict, num_output_frames):
    """
    Compute evaluation metrics
    """
    metrics = {
        'ADE': [],  # Average Displacement Error
        'FDE': [],  # Final Displacement Error
        'ADE_per_frame': [],
        'FDE_per_player': []
    }
    
    batch_size = predictions.shape[0]
    
    for b in range(batch_size):
        T = num_output_frames[b]
        player_mask = players_to_predict[b]
        
        if player_mask.sum() == 0:
            continue
        
        pred = predictions[b, player_mask, :T, :]
        target = targets[b, player_mask, :T, :]
        
        # Compute displacement errors
        displacements = torch.sqrt(((pred - target) ** 2).sum(dim=-1))
        
        # ADE: average over all frames and players
        ade = displacements.mean().item()
        metrics['ADE'].append(ade)
        
        # FDE: error at final frame
        fde = displacements[:, -1].mean().item()
        metrics['FDE'].append(fde)
        
        # Per-frame ADE
        ade_per_frame = displacements.mean(dim=0).cpu().numpy()
        metrics['ADE_per_frame'].append(ade_per_frame)
        
        # Per-player FDE
        fde_per_player = displacements[:, -1].cpu().numpy()
        metrics['FDE_per_player'].extend(fde_per_player.tolist())
    
    # Aggregate metrics
    summary = {
        'mean_ADE': np.mean(metrics['ADE']),
        'std_ADE': np.std(metrics['ADE']),
        'mean_FDE': np.mean(metrics['FDE']),
        'std_FDE': np.std(metrics['FDE']),
        'median_ADE': np.median(metrics['ADE']),
        'median_FDE': np.median(metrics['FDE'])
    }
    
    return summary, metrics


# Evaluate on validation set
print("\n📊 Evaluating model on validation set...")

model.eval()
all_metrics = []

with torch.no_grad():
    for batch in tqdm(val_loader, desc="Evaluating"):
        input_seq = batch['input_seq'].to(device)
        output_seq = batch['output_seq'].to(device)
        mask = batch['mask'].to(device)
        role_ids = batch['role_ids'].to(device)
        side_ids = batch['side_ids'].to(device)
        players_to_predict = batch['players_to_predict']
        num_output_frames = batch['num_output_frames']
        
        # Predict
        pred_traj, _ = model(input_seq, role_ids, side_ids, num_output_frames, mask)
        
        # Apply physics constraints
        pred_traj = physics.apply_constraints(pred_traj)
        
        # Compute metrics
        summary, metrics = compute_metrics(
            pred_traj.cpu(),
            output_seq.cpu(),
            players_to_predict,
            num_output_frames
        )
        
        all_metrics.append(summary)

# Aggregate all metrics
final_metrics = {
    'ADE': np.mean([m['mean_ADE'] for m in all_metrics]),
    'FDE': np.mean([m['mean_FDE'] for m in all_metrics]),
    'ADE_std': np.mean([m['std_ADE'] for m in all_metrics]),
    'FDE_std': np.mean([m['std_FDE'] for m in all_metrics])
}

print("\n" + "="*60)
print("📊 FINAL EVALUATION METRICS")
print("="*60)
print(f"Average Displacement Error (ADE): {final_metrics['ADE']:.4f} ± {final_metrics['ADE_std']:.4f} yards")
print(f"Final Displacement Error (FDE):   {final_metrics['FDE']:.4f} ± {final_metrics['FDE_std']:.4f} yards")
print("="*60)

In [ ]:
def visualize_prediction(input_seq, pred_traj, target_traj, metadata, player_idx=0):
    """
    Visualize a single player's trajectory prediction
    """
    fig, ax = plt.subplots(figsize=(15, 8))
    
    # Draw field
    ax.set_xlim(0, 120)
    ax.set_ylim(0, 53.3)
    ax.set_xlabel('X (yards)', fontsize=12)
    ax.set_ylabel('Y (yards)', fontsize=12)
    ax.set_title(f"Trajectory Prediction - Game {metadata['game_id']}, Play {metadata['play_id']}", 
                 fontsize=14, fontweight='bold')
    
    # Draw field lines
    for x in range(0, 121, 10):
        ax.axvline(x, color='white', linewidth=0.5, alpha=0.3)
    ax.axhline(53.3/2, color='white', linewidth=1, alpha=0.5)
    
    # Plot input trajectory
    input_x = input_seq[:, player_idx, 0].cpu().numpy()
    input_y = input_seq[:, player_idx, 1].cpu().numpy()
    ax.plot(input_x, input_y, 'o-', color='blue', linewidth=2, 
            markersize=6, label='Input Trajectory', alpha=0.7)
    
    # Plot predicted trajectory
    pred_x = pred_traj[player_idx, :, 0].cpu().numpy()
    pred_y = pred_traj[player_idx, :, 1].cpu().numpy()
    ax.plot(pred_x, pred_y, 's-', color='red', linewidth=2, 
            markersize=6, label='Predicted Trajectory', alpha=0.7)
    
    # Plot ground truth trajectory
    target_x = target_traj[player_idx, :, 0].cpu().numpy()
    target_y = target_traj[player_idx, :, 1].cpu().numpy()
    ax.plot(target_x, target_y, '^-', color='green', linewidth=2, 
            markersize=6, label='Ground Truth', alpha=0.7)
    
    # Mark start and end points
    ax.scatter(input_x[0], input_y[0], s=200, c='blue', marker='o', 
               edgecolors='black', linewidths=2, zorder=5, label='Start')
    ax.scatter(target_x[-1], target_y[-1], s=200, c='green', marker='*', 
               edgecolors='black', linewidths=2, zorder=5, label='End (GT)')
    ax.scatter(pred_x[-1], pred_y[-1], s=200, c='red', marker='X', 
               edgecolors='black', linewidths=2, zorder=5, label='End (Pred)')
    
    ax.legend(loc='upper right', fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.set_facecolor('#2d5016')  # Football field green
    
    plt.tight_layout()
    return fig


# Visualize some predictions
print("\n🎨 Creating visualizations...")

# Get a batch
sample_batch = next(iter(val_loader))

with torch.no_grad():
    input_seq = sample_batch['input_seq'].to(device)
    output_seq = sample_batch['output_seq']
    mask = sample_batch['mask'].to(device)
    role_ids = sample_batch['role_ids'].to(device)
    side_ids = sample_batch['side_ids'].to(device)
    num_output_frames = sample_batch['num_output_frames']
    
    pred_traj, _ = model(input_seq, role_ids, side_ids, num_output_frames, mask)
    pred_traj = physics.apply_constraints(pred_traj)

# Visualize first sample
fig = visualize_prediction(
    input_seq[0].cpu(),
    pred_traj[0].cpu(),
    output_seq[0],
    sample_batch['metadata'][0],
    player_idx=0
)
plt.savefig('trajectory_prediction_sample.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Visualization saved!")

In [ ]:
# Save final model
torch.save({
    'model_state_dict': model.state_dict(),
    'model_config': model_config,
    'feature_cols': train_dataset.feature_cols,
    'feature_means': train_dataset.feature_means,
    'feature_stds': train_dataset.feature_stds,
    'final_metrics': final_metrics
}, 'final_model_complete.pth')

# Save normalization stats
normalization_stats = {
    'feature_means': train_dataset.feature_means,
    'feature_stds': train_dataset.feature_stds,
    'feature_cols': train_dataset.feature_cols
}

with open('normalization_stats.pkl', 'wb') as f:
    pickle.dump(normalization_stats, f)

# Save metrics
with open('evaluation_metrics.json', 'w') as f:
    json.dump(final_metrics, f, indent=2)

print("\n✅ All artifacts saved!")
print("\nSaved files:")
print("  - final_model_complete.pth")
print("  - normalization_stats.pkl")
print("  - evaluation_metrics.json")
print("  - training_history.pkl")
print("  - trajectory_prediction_sample.png")